# 🧼 Data Preprocessing Techniques

Este notebook explora técnicas esenciales de **preprocesamiento de datos**, una etapa clave en cualquier proyecto de ciencia de datos o aprendizaje automático.

> El objetivo es preparar los datos para que los modelos de machine learning funcionen de forma eficiente y precisa.


# Datos de ejemplo

## 1. Importación de librerías

In [ ]:
# Importación de librerias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


## 2. Carga de datos de ejemplo

In [ ]:
# Creamos un DataFrame
data = pd.DataFrame({
    'Edad': [25, 27, np.nan, 35, 29],
    'Salario': [50000, 54000, 58000, np.nan, 62000],
    'Ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', np.nan]
})

data


## 3. Manejo de valores faltantes

Los valores faltantes pueden sesgar resultados o hacer que los modelos no funcionen. Existen varias estrategias:
- Eliminar filas/columnas con muchos valores faltantes
- Imputar (rellenar) con la media, mediana, moda, o valores específicos


In [ ]:
imputer = SimpleImputer(strategy='mean')
data[['Edad', 'Salario']] = imputer.fit_transform(data[['Edad', 'Salario']])
data


## 4. Codificación de variables categóricas

Los modelos no pueden trabajar con texto. Las variables categóricas deben transformarse en números.
- Label Encoding
- One-Hot Encoding


In [ ]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded = encoder.fit_transform(data[['Ciudad']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['Ciudad']))

# Unimos al dataset original
data = data.drop('Ciudad', axis=1)
data = pd.concat([data, encoded_df], axis=1)
data


## 5. Escalado de características numéricas

Muchas técnicas de ML se ven afectadas por la escala de los datos. Por ejemplo, KNN o regresión logística.

- `StandardScaler`: media 0, varianza 1
- `MinMaxScaler`: valores entre 0 y 1


In [ ]:
scaler = StandardScaler()
data[['Edad', 'Salario']] = scaler.fit_transform(data[['Edad', 'Salario']])
data


# Data Preprocessing Techniques with Real Data

A continuación veremos paso a paso cómo preparar datos reales para un modelo de machine learning. Usaremos el dataset de pasajeros del Titanic, que contiene datos como edad, clase social, género, etc.

> El objetivo es aplicar transformaciones necesarias para que los modelos puedan entender y procesar la información correctamente.


In [ ]:
# Librerías para manipulación de datos y visualización
import pandas as pd
import numpy as np
import seaborn as sns  # para cargar dataset y visualizar
import matplotlib.pyplot as plt

# Librerías para preprocesamiento
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [ ]:
# Cargar el dataset del Titanic desde seaborn
df = sns.load_dataset("titanic")
df.head()

In [ ]:
# Dimensiones y tipos
print(df.shape)
print(df.dtypes)

# Ver cuántos valores faltantes hay por columna
df.isnull().sum()

> Vemos que hay columnas con muchos valores faltantes, como `deck`, y otras con algunos, como `age` y `embarked`.


In [ ]:
# A. Separar variables numéricas y categóricas

# Selección básica de columnas útiles
df_model = df[['survived', 'pclass', 'sex', 'age', 'embarked']].copy()

# Identificamos variables numéricas y categóricas
numerical_features = ['age']
categorical_features = ['pclass', 'sex', 'embarked']

In [ ]:
# B. Definir transformaciones para cada tipo de dato

# Imputación de valores numéricos + escalado
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # reemplaza nulos por mediana
    ('scaler', StandardScaler())                    # estandariza la escala
])

# Imputación + one-hot encoding para categóricas
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),   # modo (valor más común)
    ('encoder', OneHotEncoder(handle_unknown='ignore'))     # one-hot para texto
])

In [ ]:
# C. Combinar todo con ColumnTransformer

# Aplicamos transformaciones diferentes a cada tipo de columna
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

In [ ]:
# Aplicamos el preprocesamiento al dataset
X = df_model.drop('survived', axis=1)
y = df_model['survived']

X_preprocessed = preprocessor.fit_transform(X)

# Convertimos el resultado a DataFrame para visualizar
X_preprocessed_df = pd.DataFrame(
    X_preprocessed.toarray() if hasattr(X_preprocessed, 'toarray') else X_preprocessed,
    columns=preprocessor.get_feature_names_out()
)

X_preprocessed_df.head()

### ✅ Resumen:

- Cómo cargar datos reales con valores faltantes y tipos mixtos
- Cómo construir pipelines de preprocesamiento con `scikit-learn`
- La importancia de separar transformaciones para numéricos y categóricos
- Aplicar `ColumnTransformer` para organizar el flujo de limpieza

Este tipo de flujo es muy común en pipelines reales, sobre todo al trabajar con `sklearn`, `MLFlow` o producción.


# (ANEXO) Exploratory Data Analysis (EDA)

Antes de cualquier transformación o modelado, es fundamental entender el dataset: su estructura, patrones, valores extremos, datos faltantes, y distribuciones.

Exploraremos:

1. Información general del dataset
2. Análisis de valores faltantes
3. Estadísticas básicas
4. Distribución de variables
5. Correlaciones y relaciones clave


In [1]:
# 1. Info general del dataset

# Número de filas y columnas
print(f"Shape: {df.shape}")

# Tipos de datos y no nulos
df.info()

NameError: name 'df' is not defined

In [ ]:
# 2. Valores faltantes visuales

import missingno as msno

# Visualización rápida de nulos
msno.matrix(df)

# Total de valores nulos por columna
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# 3. Estadísticas básicas por tipo de variable

# Variables numéricas
df.describe()

# Variables categóricas
df.describe(include='object')

In [ ]:
# 4. Distribuciones importantes (dataset del Titanic)

# Distribución de la edad
sns.histplot(df['age'], kde=True)
plt.title('Distribución de Edad')
plt.show()

# Supervivencia por sexo
sns.countplot(data=df, x='sex', hue='survived')
plt.title('Supervivencia por Sexo')
plt.show()

# Distribución de clases y supervivencia
sns.countplot(data=df, x='pclass', hue='survived')
plt.title('Supervivencia por Clase Social')
plt.show()

In [ ]:
# 5. Correlaciones entre variables numéricas

# Matriz de correlación
correlation = df.select_dtypes(include='number').corr()

sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
# 6. Otras opciones son:
# Boxplots para ver outliers por grupo
# Pairplots para ver relaciones entre múltiples variables
# Violin plots si quieres distribución + densidad
# pivot_table()	Cruce entre 2 variables
# value_counts()	Frecuencia de categorías


# Boxplot: edad por clase
sns.boxplot(data=df, x='pclass', y='age')
plt.title('Distribución de Edad por Clase')
plt.show()